<a href="https://colab.research.google.com/github/2bhapby/2bhapby.github.io/blob/master/Hands_on_ch3_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성 

>(KNeighborsClassifier 사용, weights, n_neighbors 그리드 탐색으로 시도)


In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1)
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = [{'weights' : ['uniform', 'distance'], 'n_neighbors' : [3,4,5]}]

knn_clf = KNeighborsClassifier()

grid_search = GridSearchCV(knn_clf, param_grid, cv = 3, verbose = 3)
grid_search.fit(X_train, y_train)


In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
from sklearn.metrics import accuacy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

##2. 데이터 증식(training set expansion)

>MNIST 이미지를 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어보세요. 

>그런 다음 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본을 만들어 훈련 세트에 추가하세요. 

>마지막으로 이 확장된 데이터셋에서 앞에서 차은 최선의 모델을 훈련시키고 테스트 세트에서 정확도를 측정해보세요.

In [ ]:
from scipy.ndimage.interpolation import shift

In [ ]:
def shift_image(image, dx, dy):
    image = image.reshape((28,28))
    shifted_image = shift(image, [dx, dy], cval = 0, mode = 'constant')
    return shifted_image

In [ ]:
image = X_train[1000]
shifted_image_up = shift_image(image, 0, 1)
shifted_image_down = shift_image(image, 0, -1)
shifted_image_left = shift_image(image, -1, 0)
shifted_image_right = shift_image(image, 1, 0)

plt.figure(figsize=(12,3))
plt.subplot(131)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28,28), interpolation='nearest', cmap='Greys')
plt.subplot(132)
plt.title("Original", fontsize=14)
plt.imshow(shifted_image_up.reshape(28,28), interpolation='nearest', cmap='Greys')
plt.subplot(133)
plt.title("Original", fontsize=14)
plt.imshow(shifted_image_down.reshape(28,28), interpolation='nearest', cmap='Greys')
plt.subplot(134)
plt.title("Original", fontsize=14)
plt.imshow(shifted_image_left.reshape(28,28), interpolation='nearest', cmap='Greys')
plt.subplot(135)
plt.title("Original", fontsize=14)
plt.imshow(shifted_image_right.reshape(28,28), interpolation='nearest', cmap='Greys')


In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for image in y_train]

for dx, dy in ((1,0),(-1,0),(0,1),(0,-1)):
    X_train_augmented.append(shift(image, dx, dy))
    y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [ ]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [ ]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)

In [ ]:
knn_clf.fit(X_train_augmented, y_train_augmented)

In [ ]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

##3. 타이타닉(Kaggle)

따로 구현

#스팸 분류기

- https://homl.info/spamassassin 에서 샘플 다운로드  

- 훈련 세트, 테스트 세트 분리  

- 각 메일을 특성 벡터로 변환하는 데이터 준비 파이프라인 준비
> 이 파이프 라인은 하나의 이메일을 가능한 단어의 존재 여부를 나타내는 희소벡터로 바꿔야함 (포함 유무 혹은 포함 횟수)
- 이메일 헤더 제거, 소문자 변환, 구두점 제거, 모든 URLs 주소를 'URL'로 대체, 모든 숫자를 'NUMBER'로 대체, 어간 추출(즉, 단어의 끝을 떼어냄) 등의 수행할지 여부를 제어하기 위한 하이퍼파라미터 추가

###Data Analysis

In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (("ham.tar.bz2", HAM_URL), ("spam.tar.bz2", SPAM_URL)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=SPAM_PATH)
        tar_bz2_file.close()

In [ ]:
fetch_spam_data()

In [ ]:
ham_dir = os.path.join(SPAM_PATH, 'easy_ham')
spam_dir = os.path.join(SPAM_PATH, 'spam')
ham_filenames = [name for name in sorted(os.listdir(ham_dir)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(spam_dir)) if len(name) > 20]

In [ ]:
len(ham_filenames), len(spam_filenames)

In [ ]:
import email
import email.policy

def load_email(is_spam, filename, spam_path = SPAM_PATH):
    directory = 'spam' if is_spam else 'easy_ham'
    with open(os.path.join(spam_path, directory, filename), 'rb') as f:
        return email.parser.BytesParser(policy = email.policy.default).parse(f)

In [ ]:
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]


In [ ]:
print(ham_emails[1].get_content().strip())

In [ ]:
print(spam_emails[6].get_content().strip())

메일의 형식은 이미지, 첨부파일을 갖는 multipart

In [ ]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()


In [ ]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [ ]:
structures_counter(ham_emails).most_common()

In [ ]:
structures_counter(spam_emails).most_common()

ham email의 경우 대부분 plain text, pgp 서명 존재  
spam email의 경우 html의 비중이 꽤나 높음, pgp 서명 없음

In [ ]:
for header, value in spam_emails[0].items():
    print(header, ":", value)

#####훈련, 테스트 세트 분류

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails, dtype = object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#####데이터 전처리

In [ ]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [ ]:
html_spam_emails = [email for email in X_train[y_train == 1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")

In [ ]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")

In [ ]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype == 'text/plain':
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

print(email_to_text(sample_html_spam)[:100], "...")

#####어간 추출

In [ ]:
try:
    import nltk

    stemmer = nltk.PorterStemmer() #어간추출
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, "=>", stemmer.stem(word))
except ImportError:
    print("Error")
    stemmer = None

In [ ]:
try:
    import google.colab
    !pip install -q -U urlextract

except ImportError:
    pass

In [ ]:
#url 바꾸는데 urlextract 사용

try:
    import urlextract 

    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
    print("Error: replacing URLs requires the urlextract module.")
    url_extractor = None

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ''
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key = lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+',' ', text, flags = re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [ ]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

단어 카운트를 벡터로 변환하는 transformer

In [ ]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, vocabulary_size = 1000):
        self.vocabulary_size = vocabulary_size

    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word:index + 1 for index, (word, count) in enumerate(most_common)}
        return self

    def transform(self, X, y = None):
        rows = []
        cols = []
        data = []

        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape = (len(X), self.vocabulary_size + 1))

In [ ]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors.toarray()

In [ ]:
vocab_transformer.vocabulary_

In [ ]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
        ('email_to_wordcount', EmailToWordCounterTransformer()),
        ('wordcount_to_vector', WordCounterToVectorTransformer())
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver = 'lbfgs', max_iter = 1000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

In [ ]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf = LogisticRegression(solver = 'lbfgs', max_iter=1000, random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100*precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100*recall_score(y_test, y_pred)))